# Stock Market Sentiment Analysis

In [4]:
# Pulls list of all stocks listed in the NYSE to obtain news on. There is a limit on the API on daily requests of 50 every 12 hours. Would need to upgrade to implement
#import chardet 

# use the detect method to find the encoding
# 'rb' means read in the file as binary
#with open('NYSELIST.csv', 'rb') as file:
#    print(chardet.detect(file.read()))
#file_path = 'NYSELIST.csv'
#Stocklist =  pd.read_csv("NYSELIST.csv", encoding="Windows-1252")
#Stocklist.drop(columns=[' Company'])
#print(Stocklist[" Company"])

{'encoding': 'Windows-1252', 'confidence': 0.7292082225034218, 'language': ''}


## Retrieving News

In [9]:
#library import

import requests
import pandas as pd
import json
import re
from datetime import datetime, timedelta

#Variable definition 

current_date = datetime.now().date()
date_30_days_ago = current_date - timedelta(days=30)
df_main=pd.DataFrame()
Stocklist=['Exxon','Nvidia','Intel','Tesla','Ford','Kraft Heinz']

for i in Stocklist:
    url = ('https://newsapi.org/v2/everything?'
           f'q={i}&'
           f'from=q{current_date}&'
           f'to=q{date_30_days_ago}'
           'sortBy=popularity&'
           'apiKey=')
    response = requests.get(url)
    data=response.json()
    df=pd.DataFrame(data['articles'])
    df=df.drop(columns=['source', 'author', 'description', 'url', 'urlToImage', 'publishedAt', 'content'])
    df.insert(0, 'Ticker', i)   
    df_main = pd.concat([df_main, df])
#end of for

#Cleaning the data by removing non ascii characters
pattern = re.compile(r'[^\x00-\x7F]+')
def contains_non_ascii(row):
    return any(pattern.search(str(value)) for value in row)
df_main = df_main[~df_main.apply(contains_non_ascii, axis=1)]
mask = df_main['title'].apply(lambda text: any(element in text for element in Stocklist))
df_main=df_main[mask]
df_main.to_csv('News.csv', index=False)


## Executing Rule Based Sentiment Analysis

In [10]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process


# Load the CSV file
file_path = 'News.csv'
df = pd.read_csv(file_path)
SentimentValue=0

# Define keywords for positive and negative sentiment

#original arrays
#positive_keywords = ['tops','jumps','purchase','increase','sign','giant','deal','bullish','growth', 'profit', 'success','safe','generate', 'win','won', 'gain', 'improve', 'raised', 'bought', 'buy', 'acquire','acquires', 'grew','gain','Rally', 'melt up', 'was up', 'up', 'climbed', 'Rally', 'Rallied']
#negative_keywords = ['boasts','cancel','suspension','stall','quashed','bearish','loss', 'lost','recalls', 'decline','disaster', 'problem', 'lawsuit', 'fail', 'drop', 'trimmed', 'sell', 'sold', 'cut', 'was down', 'crash', 'decrying', 'melt down', 'broken']


positive_keywords = ['tops','jumps','purchase','increase','sign','giant','deal','growth', 'profit', 'success','safe','generate', 'win','won', 'gain', 'improve', 'raised', 'bought', 'buy', 'acquire','acquires', 'grew','gain', 'was up', 'up', 'climbed']
very_positive_keywords=['upgrade','bullish', 'rally', 'melt up', 'rallied']
negative_keywords = ['boasts','cancel','suspension','stall','quashed','loss', 'lost','recalls', 'decline','disaster', 'problem', 'lawsuit', 'fail', 'drop']
very_negative_keywords=['downgrade','bearish', 'trimmed', 'sell', 'sold', 'cut', 'was down', 'crash', 'decrying', 'melt down', 'broken']

def rule_based_labeling(title):
    title = title.lower()
    title=title.split(" ")
    count_positive=0
    count_negative=0
    for word in title:
        match, similarity=process.extractOne(word, positive_keywords)
        if(similarity>=80):
            count_positive+=1
        match, similarity=process.extractOne(word, negative_keywords)
        if(similarity>=80):
            count_negative+=1            
        match, similarity=process.extractOne(word, very_positive_keywords)
        if(similarity>=80):
            count_positive+=2
        match, similarity=process.extractOne(word, very_negative_keywords)
        if(similarity>=80):
            count_negative+=2
    return count_positive-count_negative
        


# Apply rule-based labeling
df['sentiment'] = df['title'].apply(rule_based_labeling)

output_file_path = 'news_with_sentiment.csv'
df.to_csv(output_file_path, index=False)


PermissionError: [Errno 13] Permission denied: 'news_with_sentiment.csv'